In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of the website you want to fetch 
url = "https://support.apple.com/sitemap"

# Step 1: Download the HTML content
response = requests.get(url)
html_content = response.text


# Step 2: Parse HTML and extract text
soup = BeautifulSoup(html_content, "html.parser")

# Remove scripts, styles, and other non-visible elements
for element in soup(["script", "style", "header", "footer", "nav"]):
    element.decompose()

# Get only the visible text
text = soup.get_text(separator="\n", strip=True)

# Now `text` contains the webpage text
print(text[:1000])  # print first 1000 characters as a preview


If Find My is offline or not working - Apple Support
If Find My is offline or not working
Learn what to do if you don't see your device listed, you see an alert that says your device is offline, or you see an unexpected device in Find My.
I don't see my device in Find My
I see my device in Find My, but it's offline
I see a device that I don’t own or use anymore
I use a device associated with a business or educational institution
I don't see my device in Find My
Make sure that you
set up Find My
on your device. You have to set up Find My before you can see your device in the Find My app or on
iCloud.com/find
.
If your device is lost
If your device is missing,
learn how to mark your device as lost and protect your information
.
If you have your device with you
Check that you signed in to iCloud on your device using your Apple Account: Tap Settings. If you see "Sign in to your [device]," tap it and enter your Apple Account email address or phone number and password.
Make sure that you s


In [10]:
import requests
from bs4 import BeautifulSoup
import time


def extract_text_from_html(html):
    soup = BeautifulSoup(html, "html.parser")
    
    # Find the main content div
    content_div = soup.find("div", id="sections")
    if not content_div:
        return ""
    
     # Remove all tables entirely
    for table in content_div.find_all("table"):
        table.decompose()

    # Elements that should start a new line
    block_tags = {"h1", "h2", "h3", "p"}

    # Elements that are inline (flattened)
    inline_tags = {"a", "b", "i", "span", "li", "ul"}

    # Recursive function to extract text
    def recurse(element):
        texts = []
        for child in element.children:
            if child.name in block_tags:
                # Start a new line for block elements
                texts.append(recurse(child).strip())
                texts.append("\n")
            elif child.name in inline_tags:
                # Inline: flatten the tag, keep its children
                texts.append(recurse(child))
            elif child.string:
                # Text node
                texts.append(child.string)
            elif hasattr(child, "children"):
                # Other tags: recurse
                texts.append(recurse(child))
        return "".join(texts)

    extracted_text = recurse(content_div)
    
    # Clean up extra whitespace and multiple newlines
    lines = [line.strip() for line in extracted_text.split("\n") if line.strip()]
    # relace non-breaking spaces
    lines = [line.replace("\xa0", " ") for line in lines]
    return "\n".join(lines)


start_id = 102701
end_id = 102799
text = ''

# for each id in the range, download and extract text
for article_id in range(start_id, end_id + 1):
    # URL of the website you want to fetch
    url = f"https://support.apple.com/en-us/{article_id}"
    try:
        response = requests.get(url)
        html_content = response.text
        text += extract_text_from_html(html_content)
        text += "\n" 
        time.sleep(1)
    except Exception as e:
        print(f"Error fetching {url}: {e}")

with open(f"data/text2.txt", "w", encoding="utf-8") as f:
    f.write(text)

In [8]:
url = f"https://support.apple.com/en-us/102666"
try:
    response = requests.get(url)
    html_content = response.text
except Exception as e:
    print(f"Error fetching {url}: {e}")

with open(f"data/102666.html", "w", encoding="utf-8") as f:
    f.write(html_content)